In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
from nltk.corpus import stopwords 
#import nltk
#nltk.download('stopwords')
# Load the training data
train_data = pd.read_csv("train.csv")
def hash_to_list(x):
    return [ (i.strip())[1:-1] for i in x[1:-1].split(",")]
def extract_part_of_link(x):
    return [i.split('/')[2] if i!="" else "" for i in x]
train_data["hashtags"] = train_data["hashtags"].apply(hash_to_list)
train_data["urls"] = train_data["urls"].apply(hash_to_list)
train_data["mentions"] = train_data["mentions"].apply(hash_to_list)
#train_data["urls_site"] = train_data["urls"].apply(extract_part_of_link)
# Here we split our training data into trainig and testing set. This way we can estimate the evaluation of our model without uploading to Kaggle and avoid overfitting over our evaluation dataset.
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.7, test_size=0.3)

# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

# You can examine the available features using X_train.head()
# X_train.head()
t = train_data.iloc[20]["urls"]

2022-12-04 14:53:02.957745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 14:53:02.957784: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
X_train.dtypes


text               object
favorites_count     int64
followers_count     int64
statuses_count      int64
friends_count       int64
mentions           object
urls               object
verified            int64
hashtags           object
timestamp           int64
TweetID             int64
dtype: object

In [3]:
#c = train_data.groupby("urls").size().reset_index(name='counts')
from collections import Counter
columns = ["urls", "mentions", "hashtags"]
for c in columns:
    d = dict()
    for urls in train_data[c]:
        if urls[0] == "":
            continue
        for url in urls:
            if url in d.keys():
                d[url]+=1
            else:
                d[url] = 1
    res = Counter(d.values())
    print(f"Statistic for column \"{c}\" is : {res}")


Statistic for column "urls" is : Counter({1: 184815, 2: 1043, 3: 51, 4: 20, 5: 8, 7: 4, 8: 3, 9: 2, 6: 2, 18: 1, 11: 1, 10: 1})
Statistic for column "mentions" is : Counter()
Statistic for column "hashtags" is : Counter({1: 7175, 2: 2108, 3: 704, 4: 444, 5: 258, 6: 207, 7: 151, 8: 106, 9: 98, 10: 61, 11: 56, 12: 42, 13: 36, 14: 35, 15: 30, 17: 27, 16: 27, 18: 25, 24: 23, 21: 20, 20: 19, 23: 16, 22: 15, 27: 14, 30: 14, 25: 14, 33: 13, 19: 12, 26: 10, 32: 9, 40: 9, 53: 8, 39: 8, 31: 8, 41: 8, 28: 8, 59: 7, 29: 7, 43: 6, 61: 6, 46: 6, 48: 6, 47: 6, 37: 5, 57: 5, 35: 5, 45: 5, 88: 4, 42: 4, 49: 4, 52: 4, 70: 4, 242: 3, 34: 3, 54: 3, 36: 3, 63: 3, 83: 3, 130: 3, 62: 3, 75: 3, 44: 3, 458: 2, 97: 2, 93: 2, 51: 2, 112: 2, 133: 2, 205: 2, 255: 2, 79: 2, 84: 2, 131: 2, 60: 2, 153: 2, 76: 2, 119: 2, 106: 2, 248: 2, 65: 2, 58: 2, 180: 2, 87: 2, 55: 2, 38: 2, 2058: 1, 1725: 1, 1032: 1, 1376: 1, 2075: 1, 406: 1, 451: 1, 2063: 1, 827: 1, 1284: 1, 13928: 1, 550: 1, 483: 1, 5629: 1, 238: 1, 230: 1, 310

In [6]:
# We set up an Tfidf Vectorizer that will use the top 100 tokens from the tweets. We also remove stopwords.
# To do that we have to fit our training dataset and then transform both the training and testing dataset. 
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
X_train_text = vectorizer.fit_transform(X_train['text'])
X_test_text = vectorizer.transform(X_test['text'])
print(X_train_text.shape)
train_columns = ["favorites_count","followers_count", "statuses_count", "friends_count", "verified", "timestamp"]
print(X_train.loc[:, X_train.columns != 'text'].to_numpy().shape)
X_train_all = np.hstack((X_train.loc[:, train_columns].to_numpy(), X_train_text.toarray()))
X_test_all = np.hstack((X_test.loc[:, train_columns].to_numpy(), X_test_text.toarray()))

(247778, 100)
(247778, 10)


## GradientBoosting

In [8]:
# Now we can train our model. Here we chose a Gradient Boosting Regressor and we set our loss function 
reg = GradientBoostingRegressor()#reg = RandomForestRegressor() #
#reg = LinearRegression()

# We fit our model using the training data
reg.fit(X_train_all,#X_train.loc[:,train_columns] ,
        y_train)
# And then we predict the values for our testing set
y_pred = reg.predict(X_test_all)#X_test.loc[:,train_columns])
# We want to make sure that all predictions are non-negative integers
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))



Prediction error: 6.45544349332806


## Catboost

In [7]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import accuracy_score

model = CatBoostRegressor(
    random_seed=42,
)
categorical_features_indices = []
print(X_train_all.shape)
model.fit(
    X_train_all,#X_train.loc[:,train_columns], 
    y_train,
    cat_features=categorical_features_indices,
    plot=True
);

y_pred = model.predict(X_test_all)#X_test.loc[:,train_columns])
# We want to make sure that all predictions are non-negative integers
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

print("Prediction error:", mean_absolute_error(y_true=y_test, y_pred=y_pred))


(247778, 106)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.097823
0:	learn: 234.6648007	total: 23ms	remaining: 23s
1:	learn: 222.9060809	total: 39.6ms	remaining: 19.8s
2:	learn: 212.2169531	total: 58ms	remaining: 19.3s
3:	learn: 202.3483951	total: 76.1ms	remaining: 18.9s
4:	learn: 193.5494081	total: 91.2ms	remaining: 18.1s
5:	learn: 186.0327427	total: 106ms	remaining: 17.6s
6:	learn: 178.9606429	total: 122ms	remaining: 17.3s
7:	learn: 172.7116254	total: 139ms	remaining: 17.2s
8:	learn: 167.1138377	total: 154ms	remaining: 17s
9:	learn: 161.4277042	total: 168ms	remaining: 16.7s
10:	learn: 156.6430453	total: 181ms	remaining: 16.3s
11:	learn: 152.2420569	total: 196ms	remaining: 16.1s
12:	learn: 148.3856553	total: 210ms	remaining: 16s
13:	learn: 145.6270441	total: 225ms	remaining: 15.8s
14:	learn: 141.3352432	total: 239ms	remaining: 15.7s
15:	learn: 138.8831150	total: 253ms	remaining: 15.6s
16:	learn: 135.6928159	total: 269ms	remaining: 15.5s
17:	learn: 133.4698776	total: 283ms	remaining: 15.5s
18:	learn: 130.5638111	total: 2

167:	learn: 54.6561356	total: 2.65s	remaining: 13.1s
168:	learn: 54.4521649	total: 2.67s	remaining: 13.1s
169:	learn: 54.4173810	total: 2.68s	remaining: 13.1s
170:	learn: 54.2407529	total: 2.7s	remaining: 13.1s
171:	learn: 53.9018450	total: 2.71s	remaining: 13.1s
172:	learn: 53.7904086	total: 2.73s	remaining: 13s
173:	learn: 53.7570277	total: 2.74s	remaining: 13s
174:	learn: 53.7160330	total: 2.76s	remaining: 13s
175:	learn: 53.5271645	total: 2.78s	remaining: 13s
176:	learn: 53.3681895	total: 2.79s	remaining: 13s
177:	learn: 53.3365646	total: 2.81s	remaining: 13s
178:	learn: 53.3062294	total: 2.82s	remaining: 12.9s
179:	learn: 53.2242615	total: 2.84s	remaining: 12.9s
180:	learn: 53.1902535	total: 2.85s	remaining: 12.9s
181:	learn: 53.1217849	total: 2.87s	remaining: 12.9s
182:	learn: 53.0266729	total: 2.89s	remaining: 12.9s
183:	learn: 52.9248269	total: 2.9s	remaining: 12.9s
184:	learn: 52.8872106	total: 2.92s	remaining: 12.9s
185:	learn: 52.7902676	total: 2.93s	remaining: 12.8s
186:	le

335:	learn: 43.0745907	total: 5.31s	remaining: 10.5s
336:	learn: 42.9692963	total: 5.33s	remaining: 10.5s
337:	learn: 42.9047291	total: 5.34s	remaining: 10.5s
338:	learn: 42.8589339	total: 5.36s	remaining: 10.5s
339:	learn: 42.8469547	total: 5.38s	remaining: 10.4s
340:	learn: 42.7999859	total: 5.39s	remaining: 10.4s
341:	learn: 42.7420735	total: 5.41s	remaining: 10.4s
342:	learn: 42.7244871	total: 5.43s	remaining: 10.4s
343:	learn: 42.6997703	total: 5.44s	remaining: 10.4s
344:	learn: 42.6720174	total: 5.46s	remaining: 10.4s
345:	learn: 42.5861717	total: 5.48s	remaining: 10.4s
346:	learn: 42.5746595	total: 5.49s	remaining: 10.3s
347:	learn: 42.5482730	total: 5.51s	remaining: 10.3s
348:	learn: 42.3792962	total: 5.52s	remaining: 10.3s
349:	learn: 42.2426376	total: 5.54s	remaining: 10.3s
350:	learn: 42.1994714	total: 5.55s	remaining: 10.3s
351:	learn: 42.1575063	total: 5.57s	remaining: 10.3s
352:	learn: 42.0799182	total: 5.58s	remaining: 10.2s
353:	learn: 42.0037122	total: 5.6s	remaining: 

502:	learn: 36.2624784	total: 7.99s	remaining: 7.9s
503:	learn: 36.2280325	total: 8.02s	remaining: 7.89s
504:	learn: 36.1831232	total: 8.04s	remaining: 7.88s
505:	learn: 36.1586796	total: 8.06s	remaining: 7.87s
506:	learn: 36.1060002	total: 8.08s	remaining: 7.86s
507:	learn: 36.0607681	total: 8.1s	remaining: 7.85s
508:	learn: 36.0461241	total: 8.12s	remaining: 7.84s
509:	learn: 36.0156289	total: 8.14s	remaining: 7.82s
510:	learn: 36.0016504	total: 8.16s	remaining: 7.81s
511:	learn: 35.9755679	total: 8.19s	remaining: 7.81s
512:	learn: 35.9535524	total: 8.21s	remaining: 7.8s
513:	learn: 35.9127568	total: 8.23s	remaining: 7.79s
514:	learn: 35.8549470	total: 8.25s	remaining: 7.77s
515:	learn: 35.8335377	total: 8.27s	remaining: 7.76s
516:	learn: 35.8184671	total: 8.28s	remaining: 7.74s
517:	learn: 35.7780045	total: 8.3s	remaining: 7.72s
518:	learn: 35.7448655	total: 8.31s	remaining: 7.71s
519:	learn: 35.7051310	total: 8.33s	remaining: 7.69s
520:	learn: 35.6272346	total: 8.35s	remaining: 7.6

668:	learn: 32.0163173	total: 10.7s	remaining: 5.28s
669:	learn: 31.9984157	total: 10.7s	remaining: 5.26s
670:	learn: 31.9862179	total: 10.7s	remaining: 5.25s
671:	learn: 31.9419558	total: 10.7s	remaining: 5.23s
672:	learn: 31.9062781	total: 10.7s	remaining: 5.22s
673:	learn: 31.8876112	total: 10.7s	remaining: 5.2s
674:	learn: 31.8766309	total: 10.8s	remaining: 5.18s
675:	learn: 31.8352605	total: 10.8s	remaining: 5.17s
676:	learn: 31.8207563	total: 10.8s	remaining: 5.15s
677:	learn: 31.8044371	total: 10.8s	remaining: 5.13s
678:	learn: 31.7856889	total: 10.8s	remaining: 5.12s
679:	learn: 31.7454983	total: 10.8s	remaining: 5.1s
680:	learn: 31.7308647	total: 10.9s	remaining: 5.09s
681:	learn: 31.7055657	total: 10.9s	remaining: 5.07s
682:	learn: 31.6790570	total: 10.9s	remaining: 5.06s
683:	learn: 31.6569864	total: 10.9s	remaining: 5.04s
684:	learn: 31.6304865	total: 10.9s	remaining: 5.02s
685:	learn: 31.6103552	total: 10.9s	remaining: 5.01s
686:	learn: 31.5849261	total: 11s	remaining: 5s


826:	learn: 29.0468867	total: 13.1s	remaining: 2.74s
827:	learn: 29.0327083	total: 13.1s	remaining: 2.73s
828:	learn: 28.9994541	total: 13.1s	remaining: 2.71s
829:	learn: 28.9552394	total: 13.2s	remaining: 2.69s
830:	learn: 28.9395510	total: 13.2s	remaining: 2.68s
831:	learn: 28.9127902	total: 13.2s	remaining: 2.66s
832:	learn: 28.8940141	total: 13.2s	remaining: 2.65s
833:	learn: 28.8782988	total: 13.2s	remaining: 2.63s
834:	learn: 28.8715977	total: 13.2s	remaining: 2.62s
835:	learn: 28.8536268	total: 13.3s	remaining: 2.6s
836:	learn: 28.8434021	total: 13.3s	remaining: 2.58s
837:	learn: 28.8287121	total: 13.3s	remaining: 2.57s
838:	learn: 28.8047405	total: 13.3s	remaining: 2.55s
839:	learn: 28.7917560	total: 13.3s	remaining: 2.54s
840:	learn: 28.7820413	total: 13.3s	remaining: 2.52s
841:	learn: 28.7765789	total: 13.3s	remaining: 2.5s
842:	learn: 28.7326923	total: 13.4s	remaining: 2.49s
843:	learn: 28.6984374	total: 13.4s	remaining: 2.47s
844:	learn: 28.6690660	total: 13.4s	remaining: 2

986:	learn: 26.4819866	total: 15.6s	remaining: 205ms
987:	learn: 26.4752183	total: 15.6s	remaining: 189ms
988:	learn: 26.4691633	total: 15.6s	remaining: 174ms
989:	learn: 26.4576195	total: 15.6s	remaining: 158ms
990:	learn: 26.4448364	total: 15.6s	remaining: 142ms
991:	learn: 26.4364145	total: 15.7s	remaining: 126ms
992:	learn: 26.4263207	total: 15.7s	remaining: 110ms
993:	learn: 26.4231414	total: 15.7s	remaining: 94.7ms
994:	learn: 26.4195992	total: 15.7s	remaining: 78.9ms
995:	learn: 26.3913922	total: 15.7s	remaining: 63.1ms
996:	learn: 26.3610009	total: 15.7s	remaining: 47.3ms
997:	learn: 26.3465054	total: 15.8s	remaining: 31.6ms
998:	learn: 26.3420679	total: 15.8s	remaining: 15.8ms
999:	learn: 26.3344889	total: 15.8s	remaining: 0us
Prediction error: 6.714354323812753


## Finalized

In [9]:
###################################
# Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
###################################
# Load the evaluation data
eval_data = pd.read_csv("evaluation.csv")
# Transform our data into tfidf vectors
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
train_data = pd.read_csv("train.csv")
y_train = train_data['retweets_count']
X_train_text = vectorizer.fit_transform(train_data['text'])
train_data = train_data.loc[:,train_columns]

X_train_all = np.hstack((train_data.to_numpy(), X_train_text.toarray()))

# We fit our model using the training data
reg = GradientBoostingRegressor()
reg.fit(X_train_all, y_train)
X_val_text = vectorizer.transform(eval_data['text'])
eval_data_tr = eval_data.loc[:,train_columns]
X_val_all = np.hstack((eval_data_tr.to_numpy(), X_val_text.toarray()))
# Predict the number of retweets for the evaluation dataset
y_pred = reg.predict(X_val_all)
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
# Dump the results into a file that follows the required Kaggle template
with open("gbr_predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(y_pred):
        writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])


FileNotFoundError: [Errno 2] No such file or directory: 'evaluation.csv'

### Catboost

In [9]:
###################################
# Once we finalized our features and model we can train it using the whole training set and then produce prediction for the evaluating dataset
###################################
# Load the evaluation data
eval_data = pd.read_csv("evaluation.csv")
# Transform our data into tfidf vectors
vectorizer = TfidfVectorizer(max_features=100, stop_words=stopwords.words('french'))
train_data = pd.read_csv("train.csv")
y_train = train_data['retweets_count']
X_train = vectorizer.fit_transform(train_data['text'])

train_data = train_data.loc[:,train_columns]
# We fit our model using the training data
model = CatBoostRegressor(
    random_seed=42,
)
categorical_features_indices = []
model.fit(
    train_data, y_train,
    cat_features=categorical_features_indices,
    plot=True
)
X_val = vectorizer.transform(eval_data['text'])
eval_data_tr = eval_data.loc[:,train_columns]
# Predict the number of retweets for the evaluation dataset
y_pred = model.predict(eval_data_tr)
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]
# Dump the results into a file that follows the required Kaggle template
with open("catboost_predictions.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "retweets_count"])
    for index, prediction in enumerate(y_pred):
        writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.103494
0:	learn: 227.3084694	total: 19.8ms	remaining: 19.8s
1:	learn: 214.1015269	total: 31.7ms	remaining: 15.8s
2:	learn: 202.3536656	total: 41.8ms	remaining: 13.9s
3:	learn: 192.0202205	total: 52.9ms	remaining: 13.2s
4:	learn: 183.1236626	total: 64.4ms	remaining: 12.8s
5:	learn: 174.4972183	total: 76.1ms	remaining: 12.6s
6:	learn: 167.2204251	total: 88.3ms	remaining: 12.5s
7:	learn: 160.7018324	total: 101ms	remaining: 12.5s
8:	learn: 154.4313337	total: 112ms	remaining: 12.3s
9:	learn: 148.3815698	total: 122ms	remaining: 12.1s
10:	learn: 143.4898758	total: 133ms	remaining: 12s
11:	learn: 139.4225643	total: 144ms	remaining: 11.9s
12:	learn: 134.9648654	total: 154ms	remaining: 11.7s
13:	learn: 131.1704842	total: 165ms	remaining: 11.6s
14:	learn: 127.5485716	total: 176ms	remaining: 11.6s
15:	learn: 123.9356973	total: 189ms	remaining: 11.6s
16:	learn: 120.7580422	total: 201ms	remaining: 11.6s
17:	learn: 118.1206485	total: 212ms	remaining: 11.6s
18:	learn: 115.698726

161:	learn: 57.6980306	total: 1.8s	remaining: 9.31s
162:	learn: 57.6011865	total: 1.81s	remaining: 9.3s
163:	learn: 57.4410389	total: 1.82s	remaining: 9.29s
164:	learn: 57.2944158	total: 1.83s	remaining: 9.28s
165:	learn: 57.1977987	total: 1.84s	remaining: 9.27s
166:	learn: 57.1108058	total: 1.86s	remaining: 9.27s
167:	learn: 56.9330071	total: 1.87s	remaining: 9.26s
168:	learn: 56.8040478	total: 1.88s	remaining: 9.25s
169:	learn: 56.7393698	total: 1.89s	remaining: 9.24s
170:	learn: 56.5818755	total: 1.9s	remaining: 9.23s
171:	learn: 56.5040743	total: 1.92s	remaining: 9.22s
172:	learn: 56.3598442	total: 1.92s	remaining: 9.2s
173:	learn: 56.2094935	total: 1.94s	remaining: 9.19s
174:	learn: 56.0855090	total: 1.95s	remaining: 9.18s
175:	learn: 55.9812128	total: 1.96s	remaining: 9.16s
176:	learn: 55.9277675	total: 1.97s	remaining: 9.15s
177:	learn: 55.8715533	total: 1.98s	remaining: 9.14s
178:	learn: 55.7484590	total: 1.99s	remaining: 9.12s
179:	learn: 55.6219516	total: 2s	remaining: 9.11s


331:	learn: 45.0677054	total: 3.62s	remaining: 7.28s
332:	learn: 45.0463603	total: 3.63s	remaining: 7.28s
333:	learn: 45.0332898	total: 3.64s	remaining: 7.26s
334:	learn: 44.9657457	total: 3.65s	remaining: 7.25s
335:	learn: 44.9439451	total: 3.66s	remaining: 7.24s
336:	learn: 44.9213435	total: 3.67s	remaining: 7.23s
337:	learn: 44.8684566	total: 3.68s	remaining: 7.22s
338:	learn: 44.7711529	total: 3.69s	remaining: 7.21s
339:	learn: 44.7245457	total: 3.71s	remaining: 7.2s
340:	learn: 44.7153633	total: 3.72s	remaining: 7.18s
341:	learn: 44.6731840	total: 3.73s	remaining: 7.17s
342:	learn: 44.6212241	total: 3.74s	remaining: 7.16s
343:	learn: 44.5779885	total: 3.75s	remaining: 7.14s
344:	learn: 44.5126231	total: 3.76s	remaining: 7.13s
345:	learn: 44.4657576	total: 3.77s	remaining: 7.12s
346:	learn: 44.4242085	total: 3.78s	remaining: 7.11s
347:	learn: 44.3903432	total: 3.79s	remaining: 7.1s
348:	learn: 44.3519818	total: 3.8s	remaining: 7.09s
349:	learn: 44.3394519	total: 3.81s	remaining: 7.

502:	learn: 39.0364308	total: 5.43s	remaining: 5.37s
503:	learn: 39.0207314	total: 5.45s	remaining: 5.36s
504:	learn: 39.0145767	total: 5.46s	remaining: 5.35s
505:	learn: 38.9798187	total: 5.47s	remaining: 5.34s
506:	learn: 38.9305850	total: 5.48s	remaining: 5.33s
507:	learn: 38.8840633	total: 5.49s	remaining: 5.32s
508:	learn: 38.8643328	total: 5.5s	remaining: 5.31s
509:	learn: 38.8468196	total: 5.51s	remaining: 5.29s
510:	learn: 38.8238070	total: 5.52s	remaining: 5.28s
511:	learn: 38.7977392	total: 5.53s	remaining: 5.27s
512:	learn: 38.7660409	total: 5.54s	remaining: 5.26s
513:	learn: 38.7472116	total: 5.55s	remaining: 5.25s
514:	learn: 38.7242469	total: 5.56s	remaining: 5.24s
515:	learn: 38.7026812	total: 5.57s	remaining: 5.23s
516:	learn: 38.6839779	total: 5.58s	remaining: 5.22s
517:	learn: 38.6421495	total: 5.59s	remaining: 5.21s
518:	learn: 38.6209332	total: 5.6s	remaining: 5.19s
519:	learn: 38.6020252	total: 5.61s	remaining: 5.18s
520:	learn: 38.5790047	total: 5.62s	remaining: 5

675:	learn: 35.1609187	total: 7.25s	remaining: 3.48s
676:	learn: 35.1453969	total: 7.26s	remaining: 3.46s
677:	learn: 35.1379038	total: 7.27s	remaining: 3.45s
678:	learn: 35.1295658	total: 7.28s	remaining: 3.44s
679:	learn: 35.1175897	total: 7.29s	remaining: 3.43s
680:	learn: 35.1107010	total: 7.3s	remaining: 3.42s
681:	learn: 35.1009570	total: 7.31s	remaining: 3.41s
682:	learn: 35.0839237	total: 7.32s	remaining: 3.4s
683:	learn: 35.0607331	total: 7.33s	remaining: 3.39s
684:	learn: 35.0437084	total: 7.35s	remaining: 3.38s
685:	learn: 35.0189172	total: 7.36s	remaining: 3.37s
686:	learn: 34.9716612	total: 7.37s	remaining: 3.36s
687:	learn: 34.9505548	total: 7.38s	remaining: 3.35s
688:	learn: 34.9285275	total: 7.38s	remaining: 3.33s
689:	learn: 34.9140667	total: 7.39s	remaining: 3.32s
690:	learn: 34.9083112	total: 7.4s	remaining: 3.31s
691:	learn: 34.8932655	total: 7.42s	remaining: 3.3s
692:	learn: 34.8828484	total: 7.42s	remaining: 3.29s
693:	learn: 34.8541197	total: 7.43s	remaining: 3.2

848:	learn: 32.1850593	total: 9.07s	remaining: 1.61s
849:	learn: 32.1775824	total: 9.08s	remaining: 1.6s
850:	learn: 32.1665964	total: 9.09s	remaining: 1.59s
851:	learn: 32.1572649	total: 9.1s	remaining: 1.58s
852:	learn: 32.1463956	total: 9.11s	remaining: 1.57s
853:	learn: 32.1427383	total: 9.12s	remaining: 1.56s
854:	learn: 32.1310498	total: 9.13s	remaining: 1.55s
855:	learn: 32.1175689	total: 9.14s	remaining: 1.54s
856:	learn: 32.1022674	total: 9.16s	remaining: 1.53s
857:	learn: 32.0852500	total: 9.16s	remaining: 1.52s
858:	learn: 32.0757852	total: 9.18s	remaining: 1.51s
859:	learn: 32.0574366	total: 9.19s	remaining: 1.5s
860:	learn: 32.0384891	total: 9.2s	remaining: 1.48s
861:	learn: 32.0293659	total: 9.21s	remaining: 1.47s
862:	learn: 32.0263652	total: 9.22s	remaining: 1.46s
863:	learn: 32.0076752	total: 9.23s	remaining: 1.45s
864:	learn: 31.9967468	total: 9.24s	remaining: 1.44s
865:	learn: 31.9802655	total: 9.25s	remaining: 1.43s
866:	learn: 31.9541071	total: 9.26s	remaining: 1.4